In [1]:
import torch
import time

In [2]:
batch = 16
inc = 32
H = W = 256

In [3]:
test = torch.nn.Parameter(torch.randn(batch,inc,H,W,device = "cuda"))

In [4]:
def trial(targe_func,loop=100,prof_out=False):
    all_time = - time.time()
    with torch.autograd.profiler.profile(use_cuda=True) as prof:
        for _ in range(loop):
            with torch.autograd.profiler.record_function("forwarding-method"):
                up_tensor = targe_func(test)
            loss = up_tensor.sum()
            with torch.autograd.profiler.record_function("backwarding-method"):
                loss.backward()
            test.grad.zero_() 
    torch.cuda.synchronize()   
    all_time += time.time()
    for p in prof.key_averages():
        # dividing by 10000 converts to second
        if p.key == "backwarding-method":
            backward_time = p.cuda_time/10000
        if p.key == "forwarding-method":
            forward_time = p.cuda_time/10000
    if prof_out:
        return all_time,forward_time,backward_time,prof
    else:
        return all_time,forward_time,backward_time

def trialf(targe_func,loop=100):
    all_time = - time.time()
    with torch.autograd.profiler.profile(use_cuda=True) as prof:
        for _ in range(loop):
            with torch.autograd.profiler.record_function("forwarding-method"):
                up_tensor = targe_func(test)
            loss = up_tensor.sum()
    torch.cuda.synchronize()   
    all_time += time.time()
    for p in prof.key_averages():
        # dividing by 10000 converts to second
        if p.key == "forwarding-method":
            forward_time = p.cuda_time/10000
    return all_time,forward_time,prof

In [5]:
up2d = lambda x : torch.nn.functional.interpolate(x,scale_factor = (1,2), mode="bilinear")
trial(up2d)

(1.4856865406036377, 0.2578982958984375, 1.0865410302734375)

In [6]:
grid = torch.nn.functional.affine_grid(torch.eye(2,3)[None].expand(batch,-1,-1),(batch,inc,H,W)).to("cuda")

In [7]:
sp2d = lambda x : torch.nn.functional.grid_sample(x,grid, mode="bilinear")
trial(sp2d)

(0.7900350093841553, 0.14913910056877136, 0.5385482014160156)

In [8]:
import interp_function as IF

In [9]:
zo1d = lambda x: IF.ZoomF.apply(x,3,1)
trial(zo1d)

(0.4574005603790283, 0.07893392851638793, 0.2858813303222656)

In [10]:
sh1d = lambda x: IF.ShiftFunctional(x,1.8)
res = trial(sh1d,prof_out=True)
res[:3]

(0.5145611763000488, 0.12599467020225522, 0.3027990400390625)

In [11]:
with open("./hoge.txt","w") as f:
    print(res[3].key_averages().table(sort_by="cuda_time_total"),file=f)

In [12]:
test2 = torch.arange(35,device="cuda",dtype=torch.float32).view(1,1,7,5)
test2

tensor([[[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.],
          [20., 21., 22., 23., 24.],
          [25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34.]]]], device='cuda:0')

In [13]:
S = IF.Shift()

In [14]:
test2

tensor([[[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.],
          [20., 21., 22., 23., 24.],
          [25., 26., 27., 28., 29.],
          [30., 31., 32., 33., 34.]]]], device='cuda:0')

In [17]:
S(test2,-1.8)

tensor([[[[ 9.0000, 10.0000, 11.0000, 12.0000, 13.0000],
          [14.0000, 15.0000, 16.0000, 17.0000, 18.0000],
          [19.0000, 20.0000, 21.0000, 22.0000, 23.0000],
          [24.0000, 25.0000, 26.0000, 27.0000, 28.0000],
          [29.0000, 30.0000, 31.0000, 32.0000, 33.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]], device='cuda:0')